In [6]:
import json
import random
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from transformers import TFBertModel, BertTokenizer
from gensim.models import Word2Vec
import tensorflow as tf
from keras.callbacks import EarlyStopping


In [85]:
with open('intents.json') as json_file:
    data = json.load(json_file)

In [86]:
nltk.download("punkt",quiet=True)
sentences, y, labels = [], [], []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        sentences.append(nltk.word_tokenize(pattern.lower()))
        y.append(intent["tag"])
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

# Train Word2Vec model
word2vec = Word2Vec(sentences, min_count=1, vector_size=100)

# Convert text to numerical vectors
max_len = max([len(s) for s in sentences])
X = np.zeros((len(sentences), max_len, 100))
for i, sentence in enumerate(sentences):
    for j, word in enumerate(sentence):
        X[i, j] = word2vec.wv[word]

# Encode the labels
label_encoder = LabelEncoder()
y_vec = label_encoder.fit_transform(y)


In [87]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
input_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

bert_output = bert_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)
pooled_output = bert_output.pooler_output
dense = Dense(len(labels), activation="softmax")(pooled_output)

model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=dense)

# Compile and train the model
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
c:\Users\eobri\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, us

In [88]:
def tokenize(sentences, tokenizer, max_len):
    input_ids, input_mask, input_type_ids = [], [], []

    for sent in sentences:
        tokens = tokenizer(sent, padding="max_length", truncation=True, max_length=max_len)
        input_ids.append(tokens["input_ids"])
        input_mask.append(tokens["attention_mask"])
        input_type_ids.append(tokens["token_type_ids"])

    return np.array(input_ids), np.array(input_mask), np.array(input_type_ids)

X_tokenized = [" ".join(s) for s in sentences]
input_ids, input_mask, input_type_ids = tokenize(X_tokenized, tokenizer, max_len)

In [89]:
X_train_ids, X_test_ids, X_train_mask, X_test_mask, X_train_type_ids, X_test_type_ids, y_train, y_test = train_test_split(input_ids, input_mask, input_type_ids, y_vec, test_size=0.2, random_state=42
)

In [90]:
# Train the model
epochs = 50
batch_size = 8
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

history = model.fit(
    [X_train_ids, X_train_mask, X_train_type_ids],
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([X_test_ids, X_test_mask, X_test_type_ids], y_test),callbacks=[early_stopping] 
)

Epoch 1/50
9/9 [==============================] - 29s 2s/step - loss: 3.0032 - accuracy: 0.0588 - val_loss: 2.9014 - val_accuracy: 0.0588
Epoch 2/50
9/9 [==============================] - 14s 2s/step - loss: 2.7728 - accuracy: 0.1618 - val_loss: 2.7886 - val_accuracy: 0.0000e+00
Epoch 3/50
9/9 [==============================] - 13s 1s/step - loss: 2.3919 - accuracy: 0.2500 - val_loss: 2.8161 - val_accuracy: 0.1176
Epoch 4/50
9/9 [==============================] - 14s 1s/step - loss: 2.1946 - accuracy: 0.2206 - val_loss: 2.5263 - val_accuracy: 0.2353
Epoch 5/50
9/9 [==============================] - 13s 1s/step - loss: 1.9329 - accuracy: 0.4118 - val_loss: 2.5153 - val_accuracy: 0.1176
Epoch 6/50
9/9 [==============================] - 12s 1s/step - loss: 1.8092 - accuracy: 0.4559 - val_loss: 2.3519 - val_accuracy: 0.1176
Epoch 7/50
9/9 [==============================] - 13s 1s/step - loss: 1.5998 - accuracy: 0.5294 - val_loss: 2.2160 - val_accuracy: 0.2941
Epoch 8/50
9/9 [==============

In [91]:
model.save('telehealth_chatbot.h5')

In [2]:
import tensorflow as tf
from keras.models import load_model
from keras.utils import custom_object_scope
from transformers import TFBertModel

# register custom object for TFBertModel layer
with custom_object_scope({'TFBertModel': TFBertModel}):
    # load the saved model
    model = load_model('telehealth_chatbot.h5')


c:\Users\eobri\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def predict_intent(text, model, tokenizer, word2vec, label_encoder):
    tokens = nltk.word_tokenize(text.lower())
    tokenized_text = " ".join(tokens)
    input_ids, input_mask, input_type_ids = tokenize([tokenized_text], tokenizer, max_len)

    probabilities = model.predict([input_ids, input_mask, input_type_ids])[0]
    intent_idx = np.argmax(probabilities)

    return label_encoder.inverse_transform([intent_idx])[0]

In [7]:
with open('intents.json') as file:
    intent_data_chatbot = json.load(file)['intents']

def get_response(user_input):
    # predict user intent
    intent = predict_intent(user_input, model, tokenizer, word2vec, label_encoder)

    # map intent to response
    for intent_data in intent_data_chatbot:
        if intent_data['tag'] == intent:
            response = random.choice(intent_data['responses'])
            break
    
    return response

   

In [9]:

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    response = get_response(user_input)
    print(f"Bot: {response}")

NameError: name 'word2vec' is not defined